In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [11]:
examples = [  
    {
        "movie": "신과 함께-죄와벌",
        "answer": """
        Here is what I know:
        emoji: 🔥👼⚖️
        """,
    },
    {
        "movie": "범죄도시1",
        "answer": """
        Here is what I know:
        emoji: 👊🚔🕶️
        """,
    },
        {
        "movie": "굿윌헌팅",
        "answer": """
        Here is what I know:
        emoji: 🧠💬🌱
        """,
    },
]

In [35]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}")
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-famous film critic, you give short answers."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}"),
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=15,
    return_messages=True
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)

In [36]:
invoke_chain("뷰티인사이드")


        Here is what I know:
        emoji: 💄💔👩‍⚕️The human mentions "뷰티인사이드" and the AI responds with emojis related to beauty and emotions.content='\n        Here is what I know:\n        emoji: 💄💔👩\u200d⚕️'


In [37]:
invoke_chain("탑건")

Here is what I know:
emoji: ✈️🔥🕶️The human mentions "뷰티인사이드" and the AI responds with emojis related to beauty and emotions. When the human brings up "탑건," the AI responds with emojis representing a plane, fire, and sunglasses.content='Here is what I know:\nemoji: ✈️🔥🕶️'


In [ ]:
recall_prompt = ChatPromptTemplate.from_messages([
    # 대화 기록 가져오기
    MessagesPlaceholder(variable_name="history"),
    # 사용자 질문
    ("h7uman", "제가 먼저 질문한 영화는 무엇인가요?")
])

# 2-2) history 를 주입하는 체인 만들기
recall_chain = (
    RunnablePassthrough.assign(history=load_memory) | recall_prompt | chat
)

In [40]:
# 2-3) 실행
recall_result = recall_chain.invoke({})
recall_result.content

뷰티인사이드 🌸💄💅

'뷰티인사이드 🌸💄💅'